In [1]:
import numpy as np
import plotly.graph_objects as go
from scipy.interpolate import RegularGridInterpolator


# ============================================================
# Random seed
# ============================================================

rng = np.random.default_rng(0)


# ============================================================
# Grid (radians)
# ============================================================

n = 800

lat = np.linspace(-np.pi/2, np.pi/2, n)
lon = np.linspace(-np.pi, np.pi, n)

Lon, Lat = np.meshgrid(lon, lat)


# ============================================================
# CONUS mask
# ============================================================

lon_min, lon_max = np.deg2rad([-125, -66])
lat_min, lat_max = np.deg2rad([25, 49])

mask_us = (
    (Lon >= lon_min) &
    (Lon <= lon_max) &
    (Lat >= lat_min) &
    (Lat <= lat_max)
)


# ============================================================
# Haversine distance
# ============================================================

def spherical_distance(lat1, lon1, lat2, lon2):

    dlat = lat1 - lat2
    dlon = lon1 - lon2

    a = (
        np.sin(dlat/2)**2 +
        np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    )

    a = np.clip(a, 0, 1)

    return 2*np.arcsin(np.sqrt(a))


# ============================================================
# Generate emission sources
# ============================================================

H = 2000   # number of emission sources

emission_lon = rng.uniform(lon_min, lon_max, H)
emission_lat = rng.uniform(lat_min, lat_max, H)


# ============================================================
# Base air quality field
# ============================================================

aq = np.zeros_like(Lon)

# decay scale = ~60 km

sigma = 60000 / 6371000


for h in range(H):

    dist = spherical_distance(
        Lat, Lon,
        emission_lat[h], emission_lon[h]
    )

    aq += np.exp(-(dist**2)/(2*sigma**2))


# ============================================================
# Load wind speed
# ============================================================

wind = np.load("wind_data.npz")

lon_wind = wind["lon"]
lat_wind = wind["lat"]
speed_wind = wind["speed"]


lon1d = np.deg2rad(lon_wind[0])
lat1d = np.deg2rad(lat_wind[:,0])


if lat1d[1] < lat1d[0]:

    lat1d = lat1d[::-1]
    speed_wind = speed_wind[::-1]


interp_speed = RegularGridInterpolator(
    (lat1d, lon1d),
    speed_wind,
    bounds_error=False,
    fill_value=np.nan
)


points = np.stack([Lat.ravel(), Lon.ravel()], axis=1)

speed = interp_speed(points).reshape(Lat.shape)

aq_original = aq.copy()


# ============================================================
# Wind penalty
# ============================================================

speed_norm = (
    speed - np.nanmin(speed)
) / (
    np.nanmax(speed) - np.nanmin(speed)
)

speed_norm = np.nan_to_num(speed_norm)


tau = np.nanpercentile(speed_norm[mask_us], 75)

alpha = 3.0


penalty = np.maximum(speed_norm - tau, 0)

aq *= np.exp(-alpha * penalty)

aq_wind = aq.copy()


# ============================================================
# Normalize
# ============================================================

aq -= np.nanmin(aq)

aq /= np.nanmax(aq)


aq[~mask_us] = np.nan


# ============================================================
# Save emission coords for plotting later
# ============================================================

emission_lat_deg = np.rad2deg(emission_lat)
emission_lon_deg = np.rad2deg(emission_lon)

In [2]:
# flatten for Plotly
lat_deg = np.rad2deg(Lat).flatten()
lon_deg = np.rad2deg(Lon).flatten()
aq_flat = aq.flatten()

mask = ~np.isnan(aq_flat)
lat_deg = lat_deg[mask]
lon_deg = lon_deg[mask]
aq_flat = aq_flat[mask]

# Plotting with Plotly
fig = go.Figure(go.Scattergeo())

fig.add_trace(go.Scattergeo(
    lon=lon_deg, lat=lat_deg, mode="markers", showlegend=False,
    marker=dict(size=5, color=aq_flat, colorscale="Turbo", colorbar=dict(title="Air Quality"), opacity=0.6)
))


fig.add_trace(go.Scattergeo(
    lon=emission_lon_deg, lat=emission_lat_deg, mode="markers", showlegend=False,
    marker=dict(size=2, color="Black", symbol="x"), name="Emission Sources"))

fig.update_geos(
    visible=False, resolution=110, scope="usa",
    showcountries=True, countrycolor="Black",
    showsubunits=True, subunitcolor="Blue"
)
# fig.update_geos(projection_type="natural earth")

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, width=800, height=500)

fig.show()

In [3]:
diff = aq_wind - aq_original

print("diff min/max:", np.nanmin(diff), np.nanmax(diff))
print("diff abs max:", np.nanmax(np.abs(diff)))



fig = go.Figure(go.Scattergeo(
    lon=lon_deg,
    lat=lat_deg,
    mode="markers",
    marker=dict(size=10, color=diff.flatten()[mask], colorscale="RdBu", colorbar=dict(title="After - Before"), opacity=0.3)
))
fig.update_geos(scope="usa", showcountries=True, showsubunits=True)

fig.update_layout(height=500, margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

diff min/max: -4.7221324698178355 0.0
diff abs max: 4.7221324698178355


In [4]:
np.savez("aq_data.npz", lon=lon_deg, lat=lat_deg, aq=aq_flat)

In [5]:
lon_deg.shape

(14017,)

In [6]:
fig = go.Figure()

fig.add_trace(go.Scattergeo(
    lon=lon_deg, lat=lat_deg, mode="markers", name="Pollution concentration",
    marker=dict(size=8, color=aq_flat, colorscale="Turbo", opacity=0.4,
        colorbar=dict(title=dict(text="Pollution", font=dict(size=25, color="grey")),
                      tickfont=dict(size=12, color="gray"), thickness=14, len=0.6, y=0.5)), showlegend=False
))

fig.add_trace(go.Scattergeo(
    lon=emission_lon_deg, lat=emission_lat_deg, mode="markers", name="Sources",
    marker=dict(size=6, color="black", symbol="x", opacity=0.4)
))

fig.update_geos(visible=False, resolution=110, scope="usa",
    showcountries=True, countrycolor="black",
    showsubunits=True, subunitcolor="gray")

fig.update_layout(width=900, height=500,
    margin=dict(l=5, r=5, t=30, b=5),
    legend=dict(font=dict(size=20), bgcolor="rgba(255,255,255,0.8)", x=0.01, y=0.99),
    title=dict(text="Pollution and Emission Sources (CONUS)", font=dict(size=30), x=0.5, y=0.98))



fig.write_image("air_pollution_compact.pdf", scale=5)
fig.write_image("air_pollution_compact.png",scale=2)

fig.show()

ValueError: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido
